In [3]:
import pandas as pd

# Problem Description

Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads.



## First look at Data

In [4]:
ms_user_dimension = pd.read_csv('ms_user_dimension.csv')
ms_user_dimension.head(3)

,user_id,acc_id
0,1,716
1,2,749
2,3,713


In [5]:
ms_acc_dimension = pd.read_csv('ms_acc_dimension.csv')
ms_acc_dimension.head(3)

,acc_id,paying_customer
0,700,no
1,701,no
2,702,no


In [6]:
ms_download_facts = pd.read_csv('ms_download_facts.csv')
ms_download_facts.head(3)

,date,user_id,downloads
0,2020-08-24 00:00:00,1,6
1,2020-08-22 00:00:00,2,6
2,2020-08-18 00:00:00,3,2


## Firsts Tougths

* Merge tabels and with user_id find if paying or not

* Create a paying and non-paying dfs

* Merge all by date

* filter [no > yes]

## Data Analysis

In [9]:
#checking for missing values and format of columns
# print(ms_acc_dimension.info())
# print(ms_user_dimension.info())
print(ms_download_facts.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       100 non-null    object
 1   user_id    100 non-null    int64 
 2   downloads  100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB
None


There is no missing values, but date coumns are not in optimal format

date -> to_datetime

In [10]:
ms_download_facts.date = pd.to_datetime(ms_download_facts.date, format='%Y-%m-%d')
ms_download_facts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       100 non-null    datetime64[ns]
 1   user_id    100 non-null    int64         
 2   downloads  100 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 2.5 KB


Now that data is fixed lets start...
## Solution

### 1. Most logical approach

In [13]:
# Merging tables
user_pay = pd.merge(ms_user_dimension,ms_acc_dimension, on='acc_id', how='left').drop('acc_id', axis=1)
df = pd.merge(ms_download_facts,user_pay, on='user_id', how='left')

#Groupby date and paying_cust and sum the downloads
gby = df.groupby(['date','paying_customer']).downloads.sum().reset_index()

# Creating a non-paying dataset
df_no = gby[gby.paying_customer == 'no'].drop('paying_customer', axis=1).rename(columns={'downloads':'non-paying'})
# Creating a non-paying dataset
df_yes = gby[gby.paying_customer == 'yes'].drop('paying_customer', axis=1).rename(columns={'downloads':'paying'})

# Merging on date
df = pd.merge(df_no,df_yes, on='date', how='outer')

# Filter
output = df[df['non-paying'] > df['paying']]

### 2. Most efficient approach

In [33]:
# Merging tables
user_pay = pd.merge(ms_user_dimension,ms_acc_dimension, on='acc_id', how='left').drop('acc_id', axis=1)
df = pd.merge(ms_download_facts,user_pay, on='user_id', how='left')

# Pivot Table
df  = pd.pivot_table(df, values='downloads', index='date', columns='paying_customer', aggfunc='sum').reset_index().sort_values('date')

# Filter
output = df[df.no > df.yes].rename(columns={'no':'non-paying', 'yes':'paying'})

## Final Output

In [34]:
output.columns.names=['']
output

,date,non-paying,paying
1,2020-08-16,15,14
2,2020-08-17,45,9
3,2020-08-18,10,7
6,2020-08-21,32,17
